# 1. Парсинг данных

# Импорты

In [74]:
import pandas as pd
import os
from typing import Optional
import time

import pandas as pd
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException


In [16]:
vacancies = pd.read_csv("vacancies_messages_50.csv")

vacancies.shape

(50, 2)

In [2]:
api_key = os.getenv("OPENROUTER_API")

# Тут я мучаю модель 

Взял модели от xiaomi, mistralai, openai. Неограничины были только xiaomi и mistralai. И в средем лучше ответы были от xiaomi, как говорится, топ за свои деньги

In [ ]:
class VacancyFields(BaseModel):
    text_id: int
    job_title: Optional[str] = Field(
        default=None,
        description="Название должности без упоминания грейда."
    )
    company: Optional[str] = None
    salary: Optional[str] = Field(
        default=None,
        description="Нормализованная зарплата по правилам из задания."
    )
    grade: Optional[str] = Field(
        default=None,
        description="intern|junior|junior+|middle|middle+|senior|lead через запятую."
    )
    tg: Optional[str] = None

parser = PydanticOutputParser(pydantic_object=VacancyFields)

prompt = ChatPromptTemplate.from_template(
    """Ты извлекаешь структурированные данные о вакансии.
Следуй правилам:
- job_title: текст из описания, грейд убери. 
- company: только название компании.
- salary: числа без пробелов, множители к/тыс умножь на 1000. Правила оформления диапазонов/от/до/в час соблюдай. Валютую писать либо $ либо руб. и через пробел. Может начинаться с З/п. Конструкции вида "от 150000 ₽ до 350000 ₽" конвертируй в формат "150000-350000 руб.".
- grade: допустимые значения intern, junior, junior+, middle, middle+, senior, lead. Если несколько — по возрастанию через запятую.
- tg: телеграм-контакты в формате @username, через запятую и пробел при множестве. он может быть сразу в нужном формате без упоминания телеграмма
Если информация отсутствует, пиши None.

{format_instructions}

text_id: {text_id}
text: {text}
"""
).partial(format_instructions=parser.get_format_instructions())

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    model="xiaomi/mimo-v2-flash:free",
    # model='mistralai/devstral-2512:free',
    # model='openai/gpt-oss-20b:free',
    temperature=0
)



chain = prompt | llm | parser

def extract_fields(row, retries: int = 3) -> VacancyFields:
    payload = {
        "text_id": int(row["text_id"]),
        "text": row.get("text", "") or ""
    }
    delay = 1.0
    # На всякий сделал повтор отправки запросов, иногда что-то ломалось
    for _ in range(retries):
        try:
            return chain.invoke(payload)
        # Вдруг парсеры лягут
        except OutputParserException:
            continue
        # Иногда получал рейт лимит для модели у которой его не должно быть
        except Exception as e:
            msg = str(e).lower()
            if "429" in msg or "rate" in msg or "rate-limited" in msg or "rate limited" in msg:
                time.sleep(delay)
                delay *= 2
                continue
            raise
    return VacancyFields(text_id=payload["text_id"])

results = []
vac_iter = tqdm(vacancies.iterrows(), total=len(vacancies), desc="LLM parsing")
for _, row in vac_iter:
    result = extract_fields(row).model_dump()
    # Тут надо чтобы мониторить в реальном времени, что приходит он нейронки
    vac_iter.set_postfix_str(f"{result}")
    results.append(result)

parsed_df = pd.DataFrame(results)
parsed_df.to_csv("vacancies_parsed.csv", index=False)
parsed_df.head()

LLM parsing: 100%|██████████| 50/50 [01:31<00:00,  1.82s/it, {'text_id': 8, 'job_title': 'HR Manager', 'company': 'ALEX GROUP COMPANY', 'salary': '70000-90000 руб.', 'grade': 'junior', 'tg': '@danilavaren'}]                                


,text_id,job_title,company,salary,grade,tg
0,9,Python developer,Collectly,6000-9000 $,senior,@ann_gfio
1,31,Системный аналитик,АО «ГНИВЦ»,320000-370000 руб.,None,@NatalyaMaki
2,28,Database Administrator,Match Systems,3000$,senior,@lex_kertis
3,49,Руководителя отдела системного администрирования,None,3000$,lead,@ResearcherRIT
4,18,Analyst,Sirena Travel,120000-180000 руб.,None,@ann_gfio


# Тут мучаюсь сам

На рандомил снова выборку из нескольких id, посмотрим что вышло из парсинга через нейроную сеть

In [51]:
parsed_df.iloc[[2, 6, 8, 23, 26, 34, 36, 39, 42, 44]]

,text_id,job_title,company,salary,grade,tg
2,28,Database Administrator,Match Systems,3000$,senior,@lex_kertis
6,41,Системный аналитик,Платформа,180000-300000 руб.,"middle+, senior, lead",@Alexandrabogdanova_96
8,17,Go разработчик,КА Алешин Д.А.,500000 руб.,"middle, senior",@AleshinDmitry80
23,1,Full-stack разработчик,"КА ""АйтиКА""",300000-380000 руб.,"middle, senior",@artur_vv
26,30,Разработчик PowerBuilder,Outlines Tech,290000 руб.,None,@mnoita
34,35,Go Developer / WB Portal,WILDBERRIES,150000-600000 руб.,None,@hr_legoit
36,21,C# Developer,Sirena Travel,300000-400000 руб.,None,@ann_gfio
39,2,Копирайтер в сфере крипты,ALEX GROUP COMPANY,80000-140000 руб.,None,@danilavaren
42,34,Release manager,WILDBERRIES,150000-350000 руб.,None,@invader_s
44,46,Go Developer / WB Portal,WILDBERRIES,150000-600000 руб.,None,@Fl_recruiter


Тут у нас пробелма с валютой, как бы я не умолял, он все равно без пробела пишет, также иногда вместо долларов было написанно руб, и без конвертации, и выглядело это странно, ну ладно 

In [52]:
parsed_df.iloc[2]

text_id                          28
job_title    Database Administrator
company               Match Systems
salary                        3000$
grade                        senior
tg                      @lex_kertis
Name: 2, dtype: object

In [53]:
print(vacancies.loc[2, 'text'])

#вакансия #vacancy #job #senior #data #DB #database #engineer #architect #administrator #blockchain #crypto #fulltime #удаленка  #Match_Systems

Вакансия: Senior Database Administrator
Компания: Match Systems — продукты в сфере blockchain. Работа удаленно, с перспективами релокации или без перемещения (по выбору сотрудника)
Продукт : связан с разработкой программного комплекса для анализа и отслеживания блокчейн-транзакций для различных криптовалют (AML - anti money laundering) ; создание экосистем для принятия платежей и обмена криптовалют
Локация компании: Сингапур, Дубай, Москва, Казахстан 
Зарплатная вилка: от 3000$ (определяется по результатам собеседования) 
Занятость: полная (на протяжении стажировки возможна частичная)

Задачи :
- Проектирование структуры хранения данных из сети блокчейн;
- Оптимизации запросов к данным, консультации разработчиков;
- Поддержка заданного уровня производительности при росте объемов данных.
- Создание\мониторинг\поддержание ETL\ELT процессов для д

Тут окей в целом, только модель придумала валюту, допустим

In [54]:
parsed_df.iloc[6]

text_id                          41
job_title        Системный аналитик
company                   Платформа
salary           180000-300000 руб.
grade         middle+, senior, lead
tg           @Alexandrabogdanova_96
Name: 6, dtype: object

In [55]:
print(vacancies.loc[6, 'text'])

#вакансия #senior #middle+ #lead #удаленка #офис #fulltime #Platforma #SystemsAnalyst
◽️Вакансия: Системный аналитик
◽️Гражданство: РФ (локация не имеет значение) 
◽️Заработная плата: 180 000 - 300 000 на руки 
◽️Занятость: Полный день
◽️Компания: Платформа
◽️Требуемый опыт работы: более 3 лет в системном анализе

✅ Привет!
Мы разрабатываем проект, аналогов которому нет на рынке России. Разработка идёт уже 4 месяца и сейчас нас порядка 40 человек в 3 командах, планируем запустить ещё 3.

Наша задача построить платформу, которая в автоматическом режиме будет изготавливать проектную документацию для многоквартирного дома без участия человека.

✅ Требования:

- Понимание специфики строительного процесса и опыт в BIM от 3х лет;
- Опыт работы в роли IT Product/project manager продуктах;
- Практический опыт взаимодействия с командой разработки для координации сроков проекта — высоко организованы, проактивны, знаете Agile;
- Высшее строительное образование.


✅ Тех стек команды: С#/ .NET 7.0/

Также пытался заставить модель, но все равно какой-то плюс пишет, хотя должна писать `от`

In [56]:
parsed_df.iloc[20]

text_id                  12
job_title      Tech Lead QA
company           RecruitIT
salary               3000$+
grade                  lead
tg           @ResearcherRIT
Name: 20, dtype: object

In [57]:
print(vacancies.loc[20, 'text'])

#вакансия #qa #auto #fulltime #remote #vacancy

🔎 Ищем Tech Lead QA в финтех продукт 🔍

📍 УДАЛЕННО (Кипр, Казахстан, Грузия Турция)
_______________

💰ЗП: от 3000$ и выше + ДМС, техника
+ регулярный пересмотр грейда и ЗП
КА: RecruitIT 

Мы финтех компания, которая занимается разработкой платежных решений на международном рынке. 

Делаем классный платежный сервис, с возможностью как карточных переводов так и крипты.

📍На данный момент ищем Tech Lead QA auto для управление командой QA в интеграционном блоке.

☑️ Наш идеальный напарник:
* от 4х лет опыта автотестирования; 
* опыт управления командой;
* будет плюсом опыт на php или желание его освоить.

💡 Чем предстоит заниматься:
* написание планов тестирования, тест-кейсов и контрольных списков; 
* обеспечивать поддержку и руководство для тестировщиков;
* распределение задач, контроль и оценка результатов тестирования;
* писать автотесты и заглушки.

🔥Взамен приготовили для вас:

* белая ЗП + регулярные пересмотры;
* удаленный формат рабо

Тут в целом ок, зп просто нет в вакансии

In [58]:
parsed_df.iloc[30]

text_id                   43
job_title       IT Recruiter
company             AIPeople
salary                  None
grade                 junior
tg           @nastyaishiring
Name: 30, dtype: object

In [59]:
print(vacancies.loc[30, 'text'])

#junior #vacancy #вакансия #recruiter

Junior IT Recruiter
Full-time | Remote

Привет! 👋
Меня зовут Настя, я рекрутер компании AIPeople. Наша команда занимается передовыми проектами в сфере финтеха. ML и блокчейн - это наша стихия. Работаем из офисов в Петербурге, на Кипре и удаленно.
Сейчас мы ищем в нашу команду Junior IT Recruiter.
Мы готовы рассмотреть кандидатов с минимальным опытом в IT подборе, но большим желанием развиваться в этом направлении. У нас небольшая команда из 2х человек, будем рады, если ты тоже к нам присоединишься!
Спектр задач на позицию довольно стандартный. Что важнее - это твой подход к работе, самостоятельность и желание достичь результата!

Чем предстоит заниматься:
 • Снятие заявки на вакансию, формирование профиля подходящего кандидата;
 • Формирование пула кандидатов используя различные источники, первый контакт с ними;
 • Ведение кандидата по всем этапам до оффера, предоставление оперативной обратной связи;
 • Ведение базы кандидатов (у нас Huntflow);
 •

Модель придумала грейд, видимо `От 3-х лет опыта` для нее это middle или senior

In [60]:
parsed_df.iloc[23]

text_id                           1
job_title    Full-stack разработчик
company                 КА "АйтиКА"
salary           300000-380000 руб.
grade                middle, senior
tg                        @artur_vv
Name: 23, dtype: object

In [61]:
print(vacancies.loc[23, 'text'])

#вакансия #node #vue #fullstack #удаленно 

Формат работы: Удаленно, на момент оформление нужно быть в РФ
Занятость: Полная 
Компания: КА "АйтиКА" ищет для заказчика 
Зарплатная вилка: 300-380 т.р. на руки
Варианты трудоустройства: ТК РФ

Ищу Full-stack разработчика (Node.js, Vue.js, Nuxt.js) в строительную компанию, которая более 20 лет занимается постройкой элитной жилой и коммерческой недвижимости. Компания имеет it-аккредитацию. Проект - корпоративные сайты. Задачи от простых (поправить, заменить) до реализовать выборщик или калькулятор. В команде 5 человек. Всего 1 этап интервью.

Что нужно делать:
- Разрабатывать новые модули на сайтах, проводить их оптимизацию.
- Принимать участие в разработке ТЗ на совершенствование сайтов.
- Разрабатывать техническую документацию на созданные программные решения.
- Сопровождать внедрение новых решений, связанных с сайтами.

Мы ожидаем:
- От 3-х лет опыта на аналогичной должности.
- Уверенное знание и опыт разработки: Node.js, JavaScript, Vue.j

Сказать нечего, все заполнено верно

In [62]:
parsed_df.iloc[26]

text_id                            30
job_title    Разработчик PowerBuilder
company                 Outlines Tech
salary                    290000 руб.
grade                            None
tg                            @mnoita
Name: 26, dtype: object

In [63]:
print(vacancies.loc[26, 'text'])

#вакансия #удалёнка #IT
Здравствуйте! Компания Outlines Tech - IT решения для финансового сектора - в поиске Разработчика PowerBuilder
Вилка: до 290 тыс. на руки
Формат работы: удаленно

Задачи:

- Техническое проектирование и кодирование программного обеспечения;
- Чтение, рецензирование, согласование технической документации;
- Оценка трудоемкости разработки на основе технологической документации;
- Первичное тестирование разработанного продукта;
- Отражение выполняемых работ в автоматизированных системах учета;
- Исправление дефектов программного обеспечения;
- Подготовка дополнительной технической документации;
- Сопровождение разработанного программного обеспечения;
- Экспертиза существующих систем, моделей и применяемых технологий, в том числе в конфликтных, аварийных ситуациях;
- Участие в процессах внедрения и сопровождения систем (поддержка требований);
- Консультация смежных подразделений, взаимодействие по сложными вопросам;
- Работа в программе Smart, доработка.

Требования

Модель снова придумала валюту, ну тут российская компания, в целом верно. Странно что не придумала грейд, хотя в вакансии есть от 2 лет 

In [64]:
parsed_df.iloc[34]

text_id                            35
job_title    Go Developer / WB Portal
company                   WILDBERRIES
salary             150000-600000 руб.
grade                            None
tg                         @hr_legoit
Name: 34, dtype: object

In [65]:
print(vacancies.loc[34, 'text'])

#vacancy #вакансия #go #developer #wildberries #remote #fulltime

⭐️Go Developer / WB Portal⭐️
ЗП: от 150 000 до 600 000 
Компания: WILDBERRIES

Описание
Портал Продавцов - юнит со всеми продуктами WB для продавцов: личный кабинет, загрузка и хранение контента, поиск и каталог, антифрод, биллинг, оформление товара и доставка, аналитика и многое другое.
Это 1/4 всего бизнеса WB. В сервисе более 1 млн. поставщиков, ежедневный оборот - 5 млрд. рублей.

Примеры проектов:
- Маркетплейс
Команда занимается развитием способов доставки товаров по схеме FBS и DBS.
- Маркетинг
Команда занимается разработкой всех инструментов ценообразования для товаров  - цены и скидки, календарь акций, система маркетинга (реализация и поддержка всех маркетинговых инструментов).
- Контент
В этом сервисе поставщик подгружает свои товары на сайт. Основные функции контента - создание карточек, медиасервис, взаимодействие со смежной командой, аналитика, качество данных и рейтинг товаров/продавцов.
- Коммуникация прод

Тут окей

In [66]:
parsed_df.iloc[36]

text_id                      21
job_title          C# Developer
company           Sirena Travel
salary       300000-400000 руб.
grade                      None
tg                    @ann_gfio
Name: 36, dtype: object

In [67]:
print(vacancies.loc[36, 'text'])

#job #vacancy #sirenatravel #developer #backend

🌟 Вакансия: C# Developer
🏢 Компания: Sirena Travel
🔝 Продукт: Mixvel
🌐 Сайт: Mixvel
📍 Город: Москва
💰 Зарплатная вилка: 300-400 тыс. руб.
💼 Формат работы: Удаленный
🚀 О компании:
Sirena Travel - признанный лидер на рынке разработки и внедрения IT систем для авиаперевозчиков и туристических агентств.

🔝 Основной продукт: Mixvel - полностью автоматизированная B2B highload система бронирования, которая агрегирует весь контент travel рынка, включая расписание рейсов, стоимость билетов и многое другое. Mixvel также предоставляет аналитику на основе истории предыдущих поисков и взаимодействий клиента с агентством. Наши клиенты включают такие крупные компании, как Ozon.travel, OneTwoTrip и Kupibilet.

🌟 В связи с активным развитием продукта, разыскивается самостоятельный С# Developer, готовый внести свой вклад в команду.

🔸 Технологический стек и используемые инструменты:
.NET Core, C#, ASP.NET Web API, SQL
Архитектура: микросервисы

🔸 Мы ожида

Тут тоже, хотя вакансия сомнительная 

In [68]:
parsed_df.iloc[39]

text_id                              2
job_title    Копирайтер в сфере крипты
company             ALEX GROUP COMPANY
salary               80000-140000 руб.
grade                             None
tg                        @danilavaren
Name: 39, dtype: object

In [69]:
print(vacancies.loc[39, 'text'])

#copywriter #вакансия  #fulltime #удаленка  #remote #crypto #blockchain

Компания: ALEX GROUP COMPANY
Вилка: от 80 000 до 140 000 руб.
Вакансия: Копирайтер в сфере крипты

Мы в AlexGroup специализируемся на создании и переводе контента носителями на криптовалютную тематику и многие другие. 

Ищем копирайтера (менеджер в TG, linkedin, reddit) в крипто сфере

Требования:
— Уверенное владение английским языком: разговорный и письменный;
— Опыт работы с крипто, блокчейн и финтех проектами обязателен;
— Написание уникальных текстов  SEO-оптимизация, форматированные заголовками, абзацами, списками.;
— Составление официальных документов (питч-деки, WP, прочее).
— Подготовка любого рода текстовых материалов: статей, в том числе аналитических, пресс-релизов, SMM текстов, продающих текстов, интервью и так дале

Что мы предлагаем:
— Fulltime-работа. Возможен гибкий график (с сохранением продуктивности)
— Конкурентная ЗП от 80 000 до 140 000 р., подробнее обговорим лично
— Работа в одной из топовы

Тут в целом окей, ток в этот раз после слеша вакансию обрезали почему-то, ну ладно

In [70]:
parsed_df.iloc[42]

text_id                      34
job_title       Release manager
company             WILDBERRIES
salary       150000-350000 руб.
grade                      None
tg                   @invader_s
Name: 42, dtype: object

In [71]:
print(vacancies.loc[42, 'text'])

#vacancy #вакансия #ios #remote 

⭐️ Release manager / Основное мобильное приложение⭐️
ЗП: от 150 000 ₽ до  350 000 ₽
Компания: WILDBERRIES

Описание
Мы ищем релиз менеджера в команду iOS-разработки основного мобильного приложения WB (4.9 рейтинг в AppStore).

Ключевые требования ↩️
- Более двух лет работает на должности релиз-менеджера / проджект-менеджера;
- Обладаешь базовыми знаниями в iOS-разработке;
- Умеешь контролировать релизный цикл и следить за дедлайнами;
- Готов предлагать идеи по оптимизации процессов;

↪️ Основные задачи
- Планирование этапов жизненного цикла релизов;
- Управление релизной политикой;
- Контроль реализации и выпуска фичей в рамках запланированных сроков;
- Взаимодействие с командами (QA, iOS, Backend и т.д.);
Компенсация/мотивационный пакет
- Полная удаленка, гибрид или офис с диванчиками, гамаками, качелями, кафе и зелеными зонами. Мы находимся на метро Парк Культуры;
- Бесплатное неограниченное питание в офисах компании;
- Современный стек, все команды 

Дописала, что великие рубли в зарплате будут

In [72]:
parsed_df.iloc[44]

text_id                            46
job_title    Go Developer / WB Portal
company                   WILDBERRIES
salary             150000-600000 руб.
grade                            None
tg                      @Fl_recruiter
Name: 44, dtype: object

In [73]:
print(vacancies.loc[44, 'text'])

#vacancy #вакансия #go #developer #wildberries #remote #fulltime

⭐️Go Developer / WB Portal⭐️
ЗП: от 150 000 до 600 000 
Компания: WILDBERRIES

Описание
Портал Продавцов - юнит со всеми продуктами WB для продавцов: личный кабинет, загрузка и хранение контента, поиск и каталог, антифрод, биллинг, оформление товара и доставка, аналитика и многое другое.
Это 1/4 всего бизнеса WB. В сервисе более 1 млн. поставщиков, ежедневный оборот - 5 млрд. рублей.

Примеры проектов:
- Маркетплейс
Команда занимается развитием способов доставки товаров по схеме FBS и DBS.
- Маркетинг
Команда занимается разработкой всех инструментов ценообразования для товаров  - цены и скидки, календарь акций, система маркетинга (реализация и поддержка всех маркетинговых инструментов).
- Контент
В этом сервисе поставщик подгружает свои товары на сайт. Основные функции контента - создание карточек, медиасервис, взаимодействие со смежной командой, аналитика, качество данных и рейтинг товаров/продавцов.
- Коммуникация прод